In [1]:
import pandas as pd
import numpy as np

In [2]:
base_df = pd.read_pickle('/opt/ml/input/data/catboost_data.pkl')
base_df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,kind,uidIdx,assIdx,testIdx,...,problem_elapsed_time,prob_elapsed_time,normalized_p_elapsed_time,user_prob_elapsed_time_aver,test_prob_elapsed_time_aver,BC_prob_elapsed_time_aver,MC_prob_elapsed_time_aver,KT_prob_elapsed_time_aver,user_kt_ans_rate,normalized_solvesec
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,train,0,5354,975,...,0 days 00:00:03,3.0,-0.214734,31835.736559,66984.085333,50771.848871,115942.530782,108.744000,NaN,-0.214604
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,train,0,5355,975,...,0 days 00:00:08,8.0,-0.214700,31835.736559,66984.085333,50771.848871,115942.530782,73379.598667,NaN,-0.214584
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,train,0,5356,975,...,0 days 00:00:07,7.0,-0.214707,31835.736559,66984.085333,50771.848871,115942.530782,73379.598667,1.00,-0.214550
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,train,0,5357,975,...,0 days 00:00:07,7.0,-0.214707,31835.736559,66984.085333,50771.848871,115942.530782,73379.598667,1.00,-0.214557
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,train,0,5358,975,...,0 days 00:00:11,11.0,-0.214680,31835.736559,66984.085333,50771.848871,115942.530782,73379.598667,1.00,-0.214557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2526695,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,train,7441,2373,456,...,76 days 18:16:18,6632178.0,2.474874,829066.000000,101135.125417,58055.209790,50773.273604,74165.137364,0.25,-0.333342
2526696,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,train,7441,3909,748,...,0 days 00:00:11,11.0,-0.353570,829066.000000,76166.524853,62564.013469,46015.344786,68701.383147,NaN,2.666667
2526697,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,train,7441,3910,748,...,0 days 00:00:46,46.0,-0.353555,829066.000000,76166.524853,62564.013469,46015.344786,68701.383147,1.00,-0.333348
2526698,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,train,7441,3911,748,...,0 days 00:01:13,73.0,-0.353544,829066.000000,76166.524853,62564.013469,46015.344786,68701.383147,1.00,-0.333332


In [13]:
print(len(base_df.columns))
base_df.columns

51


Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'kind', 'uidIdx', 'assIdx', 'testIdx',
       'user_correct_answer', 'user_total_answer', 'user_acc', 'month', 'day',
       'hour', 'dayname', 'big_category', 'problem_num', 'mid_category',
       'test_mean', 'test_std', 'test_sum', 'tag_mean', 'tag_std', 'tag_sum',
       'Timestamp2', 'solvetime', 'solvesec', 'solvesec_3600', 'time_category',
       'solvesec_cumsum', 'solvecumsum_category', 'big_category_acc',
       'big_category_std', 'big_category_cumconut', 'big_category_user_acc',
       'big_category_user_std', 'big_category_answer',
       'big_category_answer_log1p', 'problem_elapsed_time',
       'prob_elapsed_time', 'normalized_p_elapsed_time',
       'user_prob_elapsed_time_aver', 'test_prob_elapsed_time_aver',
       'BC_prob_elapsed_time_aver', 'MC_prob_elapsed_time_aver',
       'KT_prob_elapsed_time_aver', 'user_kt_ans_rate', 'normalized_solvesec',
       'answerCode2'],

In [11]:
rref = base_df.copy()

rref.loc[rref['answerCode'] == -1, 'answerCode'] = 0.5
base_df['answerCode2'] = rref['answerCode']
base_df[['answerCode', 'answerCode2']]

,answerCode,answerCode2
0,1,1.0
1,1,1.0
2,1,1.0
3,1,1.0
4,1,1.0
...,...,...
2526695,0,0.0
2526696,1,1.0
2526697,1,1.0
2526698,1,1.0


In [40]:
groupby_tag = base_df.groupby("time_category")["answerCode2"].apply(list)

# 문제별로 맞은 사람 수, 틀린 사람수, 정답률 계산
correction = list() # [[문제ID, 맞은사람수, 틀린사람수, 정답률], ...]
for problem, correctness in zip(groupby_tag.index, groupby_tag):
    correction.append([problem, sum(correctness), len(correctness) - sum(correctness), round(sum(correctness) / len(correctness) * 100, 2)])

# 위에서 생성한 list를 이용해서 dataframe을 생성
tag_df = pd.DataFrame(data=correction, columns=["time_category", "correctCnt", "incorrectCnt", "correctRatio__by_TC"])
tag_df

,time_category,correctCnt,incorrectCnt,correctRatio__by_TC
0,0,9078.5,6643.5,57.74
1,1,89998.0,274749.0,24.67
2,2,34027.5,35148.5,49.19
3,3,67942.5,34261.5,66.48
4,4,758729.0,250174.0,75.20
5,5,388459.5,163766.5,70.34
6,6,18687.0,8697.0,68.24
7,7,11692.0,5071.0,69.75
8,8,5416.0,2144.0,71.64
9,9,269930.0,92085.0,74.56


In [41]:
tag_df.drop(['correctCnt','incorrectCnt'],axis=1,inplace=True)
tag_df

,time_category,correctRatio__by_TC
0,0,57.74
1,1,24.67
2,2,49.19
3,3,66.48
4,4,75.20
5,5,70.34
6,6,68.24
7,7,69.75
8,8,71.64
9,9,74.56


In [27]:
modi_df = pd.merge(base_df, tag_df, how='left', on='userID')
modi_df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,kind,uidIdx,assIdx,testIdx,...,normalized_p_elapsed_time,user_prob_elapsed_time_aver,test_prob_elapsed_time_aver,BC_prob_elapsed_time_aver,MC_prob_elapsed_time_aver,KT_prob_elapsed_time_aver,user_kt_ans_rate,normalized_solvesec,answerCode2,correctRatio__by_user
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,train,0,5354,975,...,-0.214734,31835.736559,66984.085333,50771.848871,115942.530782,108.744000,NaN,-0.214604,1.0,63.09
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,train,0,5355,975,...,-0.214700,31835.736559,66984.085333,50771.848871,115942.530782,73379.598667,NaN,-0.214584,1.0,63.09
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,train,0,5356,975,...,-0.214707,31835.736559,66984.085333,50771.848871,115942.530782,73379.598667,1.00,-0.214550,1.0,63.09
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,train,0,5357,975,...,-0.214707,31835.736559,66984.085333,50771.848871,115942.530782,73379.598667,1.00,-0.214557,1.0,63.09
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,train,0,5358,975,...,-0.214680,31835.736559,66984.085333,50771.848871,115942.530782,73379.598667,1.00,-0.214557,1.0,63.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2526695,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,train,7441,2373,456,...,2.474874,829066.000000,101135.125417,58055.209790,50773.273604,74165.137364,0.25,-0.333342,0.0,55.56
2526696,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,train,7441,3909,748,...,-0.353570,829066.000000,76166.524853,62564.013469,46015.344786,68701.383147,NaN,2.666667,1.0,55.56
2526697,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,train,7441,3910,748,...,-0.353555,829066.000000,76166.524853,62564.013469,46015.344786,68701.383147,1.00,-0.333348,1.0,55.56
2526698,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,train,7441,3911,748,...,-0.353544,829066.000000,76166.524853,62564.013469,46015.344786,68701.383147,1.00,-0.333332,1.0,55.56


In [42]:
modi_df = pd.merge(modi_df, tag_df, how='left', on='time_category')
modi_df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,kind,uidIdx,assIdx,testIdx,...,KT_prob_elapsed_time_aver,user_kt_ans_rate,normalized_solvesec,answerCode2,correctRatio__by_user,correctRatio__by_test_paper,correctRatio__by_tag,correctRatio__by_prob,correctRatio__by_BC,correctRatio__by_TC
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,train,0,5354,975,...,108.744000,NaN,-0.214604,1.0,63.09,95.27,95.73,98.40,71.22,57.74
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,train,0,5355,975,...,73379.598667,NaN,-0.214584,1.0,63.09,95.27,91.71,96.80,71.22,24.67
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,train,0,5356,975,...,73379.598667,1.00,-0.214550,1.0,63.09,95.27,91.71,91.60,71.22,66.48
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,train,0,5357,975,...,73379.598667,1.00,-0.214557,1.0,63.09,95.27,91.71,97.20,71.22,49.19
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,train,0,5358,975,...,73379.598667,1.00,-0.214557,1.0,63.09,95.27,91.71,94.80,71.22,49.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2526695,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,train,7441,2373,456,...,74165.137364,0.25,-0.333342,0.0,55.56,66.60,69.49,44.67,70.22,75.20
2526696,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,train,7441,3909,748,...,68701.383147,NaN,2.666667,1.0,55.56,65.38,69.99,64.33,68.02,74.56
2526697,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,train,7441,3910,748,...,68701.383147,1.00,-0.333348,1.0,55.56,65.38,69.99,64.00,68.02,75.20
2526698,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,train,7441,3911,748,...,68701.383147,1.00,-0.333332,1.0,55.56,65.38,69.99,78.67,68.02,75.20


In [43]:
modi_df.to_pickle('/opt/ml/input/data/catboost_data_2.pkl')

In [44]:
modi_df.columns

Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'kind', 'uidIdx', 'assIdx', 'testIdx',
       'user_correct_answer', 'user_total_answer', 'user_acc', 'month', 'day',
       'hour', 'dayname', 'big_category', 'problem_num', 'mid_category',
       'test_mean', 'test_std', 'test_sum', 'tag_mean', 'tag_std', 'tag_sum',
       'Timestamp2', 'solvetime', 'solvesec', 'solvesec_3600', 'time_category',
       'solvesec_cumsum', 'solvecumsum_category', 'big_category_acc',
       'big_category_std', 'big_category_cumconut', 'big_category_user_acc',
       'big_category_user_std', 'big_category_answer',
       'big_category_answer_log1p', 'problem_elapsed_time',
       'prob_elapsed_time', 'normalized_p_elapsed_time',
       'user_prob_elapsed_time_aver', 'test_prob_elapsed_time_aver',
       'BC_prob_elapsed_time_aver', 'MC_prob_elapsed_time_aver',
       'KT_prob_elapsed_time_aver', 'user_kt_ans_rate', 'normalized_solvesec',
       'answerCode2', 

In [48]:
a = [6.3167528, 2.00610833,4.20615735,0.79063942,1.8176827, 0.75129952
,6.35050483,0.69803887,0.16693451,0.25304803,4.86598704,1.55561293
,0.73150024,0.27796456,0.1920731, 0.26657817,0.03739622,0.07621877
,1.83836839,1.38919614,4.48349987,0.41620091,0.18533285,1.61967136
,1.26026111,1.1147534,14.34891821,8.18380788,5.81762606,2.84295719
,0.53933533,0.42409077,0.02952645, 21.89355841,1.02388778,1.22851048]

b = ['userID', 'assessmentItemID', 'testId', 'KnowledgeTag', 'user_correct_answer', 'user_total_answer', 'user_acc', 'month', 'hour', 'dayname', 'big_category', 'problem_num', 'mid_category', 'test_mean', 'test_std', 'test_sum', 'tag_mean', 'tag_std', 'solvesec', 'solvesec_3600', 'time_category', 'solvesec_cumsum', 'solvecumsum_category', 'big_category_acc', 'big_category_std', 'big_category_cumconut', 'big_category_user_acc', 'big_category_user_std', 'big_category_answer', 'big_category_answer_log1p', 'correctRatio__by_user', 'correctRatio__by_test_paper', 'correctRatio__by_tag', 'correctRatio__by_prob', 'correctRatio__by_BC', 'correctRatio__by_TC']

c = list(zip(a,b))
c.sort()

In [50]:
c.sort(reverse=True)
c

[(21.89355841, 'correctRatio__by_prob'),
 (14.34891821, 'big_category_user_acc'),
 (8.18380788, 'big_category_user_std'),
 (6.35050483, 'user_acc'),
 (6.3167528, 'userID'),
 (5.81762606, 'big_category_answer'),
 (4.86598704, 'big_category'),
 (4.48349987, 'time_category'),
 (4.20615735, 'testId'),
 (2.84295719, 'big_category_answer_log1p'),
 (2.00610833, 'assessmentItemID'),
 (1.83836839, 'solvesec'),
 (1.8176827, 'user_correct_answer'),
 (1.61967136, 'big_category_acc'),
 (1.55561293, 'problem_num'),
 (1.38919614, 'solvesec_3600'),
 (1.26026111, 'big_category_std'),
 (1.22851048, 'correctRatio__by_TC'),
 (1.1147534, 'big_category_cumconut'),
 (1.02388778, 'correctRatio__by_BC'),
 (0.79063942, 'KnowledgeTag'),
 (0.75129952, 'user_total_answer'),
 (0.73150024, 'mid_category'),
 (0.69803887, 'month'),
 (0.53933533, 'correctRatio__by_user'),
 (0.42409077, 'correctRatio__by_test_paper'),
 (0.41620091, 'solvesec_cumsum'),
 (0.27796456, 'test_mean'),
 (0.26657817, 'test_sum'),
 (0.25304803, 

In [73]:
aa = [7.23734208,1.93199325,4.87771623,2.37081064,0.83363196,8.53380287
,0.92797573,5.52197238,1.86948262,0.91899739,3.7660144, 3.89265841
,0.34074661,1.69249746,2.02244933, 12.70735602,7.84361943,6.66876869
,2.66489573,0.98102947, 20.11703448,1.16112807,1.11807677]

bb = ['userID', 'assessmentItemID', 'testId', 'user_correct_answer', 'user_total_answer', 'user_acc', 'month', 'big_category', 'problem_num', 'mid_category', 'solvesec_3600', 'time_category', 'big_category_acc', 'big_category_std', 'big_category_cumconut', 'big_category_user_acc', 'big_category_user_std', 'big_category_answer', 'big_category_answer_log1p', 'correctRatio__by_test_paper', 'correctRatio__by_prob', 'correctRatio__by_BC', 'correctRatio__by_TC']

cc = list(zip(aa,bb))
cc.sort(reverse=True)
cc

[(20.11703448, 'correctRatio__by_prob'),
 (12.70735602, 'big_category_user_acc'),
 (8.53380287, 'user_acc'),
 (7.84361943, 'big_category_user_std'),
 (7.23734208, 'userID'),
 (6.66876869, 'big_category_answer'),
 (5.52197238, 'big_category'),
 (4.87771623, 'testId'),
 (3.89265841, 'time_category'),
 (3.7660144, 'solvesec_3600'),
 (2.66489573, 'big_category_answer_log1p'),
 (2.37081064, 'user_correct_answer'),
 (2.02244933, 'big_category_cumconut'),
 (1.93199325, 'assessmentItemID'),
 (1.86948262, 'problem_num'),
 (1.69249746, 'big_category_std'),
 (1.16112807, 'correctRatio__by_BC'),
 (1.11807677, 'correctRatio__by_TC'),
 (0.98102947, 'correctRatio__by_test_paper'),
 (0.92797573, 'month'),
 (0.91899739, 'mid_category'),
 (0.83363196, 'user_total_answer'),
 (0.34074661, 'big_category_acc')]

In [74]:
c_23 = cc

In [75]:
c_23

[(20.11703448, 'correctRatio__by_prob'),
 (12.70735602, 'big_category_user_acc'),
 (8.53380287, 'user_acc'),
 (7.84361943, 'big_category_user_std'),
 (7.23734208, 'userID'),
 (6.66876869, 'big_category_answer'),
 (5.52197238, 'big_category'),
 (4.87771623, 'testId'),
 (3.89265841, 'time_category'),
 (3.7660144, 'solvesec_3600'),
 (2.66489573, 'big_category_answer_log1p'),
 (2.37081064, 'user_correct_answer'),
 (2.02244933, 'big_category_cumconut'),
 (1.93199325, 'assessmentItemID'),
 (1.86948262, 'problem_num'),
 (1.69249746, 'big_category_std'),
 (1.16112807, 'correctRatio__by_BC'),
 (1.11807677, 'correctRatio__by_TC'),
 (0.98102947, 'correctRatio__by_test_paper'),
 (0.92797573, 'month'),
 (0.91899739, 'mid_category'),
 (0.83363196, 'user_total_answer'),
 (0.34074661, 'big_category_acc')]

In [72]:
c_24

[(20.43346922, 'correctRatio__by_prob'),
 (13.50050827, 'big_category_user_acc'),
 (7.64471799, 'user_acc'),
 (7.5307173, 'big_category_user_std'),
 (6.96995349, 'userID'),
 (5.70431815, 'big_category_answer'),
 (5.66265544, 'big_category'),
 (4.97756999, 'testId'),
 (4.14673475, 'big_category_answer_log1p'),
 (3.48193445, 'solvesec_3600'),
 (2.86248948, 'time_category'),
 (2.07988205, 'correctRatio__by_TC'),
 (2.07924283, 'user_correct_answer'),
 (1.99688049, 'big_category_cumconut'),
 (1.9723526, 'assessmentItemID'),
 (1.70807739, 'problem_num'),
 (1.64962142, 'big_category_std'),
 (1.16003442, 'user_total_answer'),
 (0.89963075, 'correctRatio__by_test_paper'),
 (0.77808051, 'month'),
 (0.77238496, 'mid_category'),
 (0.67788756, 'big_category_acc'),
 (0.66862184, 'correctRatio__by_BC'),
 (0.64223465, 'solvesec_cumsum')]

In [69]:
c_26

[(19.8727106, 'correctRatio__by_prob'),
 (13.86375734, 'big_category_user_acc'),
 (7.82614177, 'user_acc'),
 (7.58845031, 'big_category_user_std'),
 (7.20765386, 'big_category_answer'),
 (6.79912164, 'userID'),
 (5.83809299, 'big_category'),
 (4.94548398, 'testId'),
 (3.83526196, 'solvesec_3600'),
 (3.834584, 'time_category'),
 (2.61912048, 'user_correct_answer'),
 (1.96902096, 'big_category_cumconut'),
 (1.91342846, 'assessmentItemID'),
 (1.87554146, 'problem_num'),
 (1.86634795, 'big_category_answer_log1p'),
 (1.34215524, 'correctRatio__by_TC'),
 (1.1552396, 'big_category_std'),
 (0.87790803, 'month'),
 (0.87037539, 'correctRatio__by_BC'),
 (0.84724031, 'mid_category'),
 (0.7090187, 'solvesec_cumsum'),
 (0.69645721, 'user_total_answer'),
 (0.46128375, 'correctRatio__by_test_paper'),
 (0.44407197, 'big_category_acc'),
 (0.39336971, 'test_mean'),
 (0.34816234, 'test_sum')]

In [65]:
c_27

[(20.18641254, 'correctRatio__by_prob'),
 (12.98528512, 'big_category_user_acc'),
 (8.09171897, 'big_category_answer'),
 (8.00481212, 'user_acc'),
 (7.89267348, 'big_category_user_std'),
 (6.90435318, 'userID'),
 (5.30582479, 'big_category'),
 (4.8203665, 'testId'),
 (3.85655657, 'time_category'),
 (3.28723971, 'solvesec_3600'),
 (2.48710708, 'big_category_answer_log1p'),
 (2.34773095, 'user_correct_answer'),
 (2.31408209, 'big_category_std'),
 (1.93553803, 'assessmentItemID'),
 (1.72425042, 'problem_num'),
 (1.56330387, 'big_category_cumconut'),
 (1.16802795, 'correctRatio__by_TC'),
 (1.07634704, 'user_total_answer'),
 (0.80427303, 'mid_category'),
 (0.62342515, 'month'),
 (0.49879314, 'correctRatio__by_BC'),
 (0.47695743, 'solvesec_cumsum'),
 (0.41584972, 'correctRatio__by_test_paper'),
 (0.37557013, 'test_mean'),
 (0.34493179, 'test_sum'),
 (0.25826767, 'big_category_acc'),
 (0.25030154, 'solvecumsum_category')]

In [63]:
c_29

[(19.77097074, 'correctRatio__by_prob'),
 (13.18047437, 'big_category_user_acc'),
 (8.06580168, 'user_acc'),
 (7.36657393, 'big_category_user_std'),
 (7.12775023, 'userID'),
 (6.83845621, 'big_category_answer'),
 (5.05912711, 'big_category'),
 (4.8555187, 'testId'),
 (3.5475294, 'solvesec_3600'),
 (3.49244107, 'time_category'),
 (3.46939977, 'big_category_answer_log1p'),
 (2.56417254, 'user_correct_answer'),
 (1.82225054, 'problem_num'),
 (1.7363002, 'big_category_std'),
 (1.72796192, 'assessmentItemID'),
 (1.59763732, 'big_category_cumconut'),
 (1.35425019, 'correctRatio__by_BC'),
 (1.24554779, 'correctRatio__by_TC'),
 (0.9968818, 'user_total_answer'),
 (0.80097806, 'mid_category'),
 (0.69718361, 'month'),
 (0.52436923, 'solvesec_cumsum'),
 (0.51127192, 'test_mean'),
 (0.41991297, 'test_sum'),
 (0.35643873, 'correctRatio__by_test_paper'),
 (0.27720876, 'big_category_acc'),
 (0.21879723, 'solvecumsum_category'),
 (0.18921305, 'dayname'),
 (0.18558093, 'hour')]

In [61]:
c_30_lr_007

[(32.61049095, 'correctRatio__by_prob'),
 (19.42521251, 'big_category_user_acc'),
 (11.52717313, 'big_category_user_std'),
 (5.80377824, 'time_category'),
 (5.70309429, 'big_category'),
 (3.76977976, 'testId'),
 (3.65887589, 'userID'),
 (2.15982356, 'correctRatio__by_BC'),
 (1.97589041, 'solvesec'),
 (1.92899232, 'assessmentItemID'),
 (1.87396745, 'big_category_acc'),
 (1.8520029, 'solvesec_3600'),
 (1.79931028, 'correctRatio__by_TC'),
 (1.19508359, 'big_category_std'),
 (0.74873297, 'user_acc'),
 (0.57047886, 'user_correct_answer'),
 (0.48888553, 'mid_category'),
 (0.37333426, 'correctRatio__by_test_paper'),
 (0.33613377, 'big_category_answer'),
 (0.32006349, 'big_category_answer_log1p'),
 (0.29591994, 'test_mean'),
 (0.29137268, 'solvesec_cumsum'),
 (0.26634062, 'month'),
 (0.19969744, 'problem_num'),
 (0.17707169, 'user_total_answer'),
 (0.13423146, 'test_sum'),
 (0.10223499, 'solvecumsum_category'),
 (0.09650487, 'hour'),
 (0.08874594, 'big_category_cumconut'),
 (0.08009807, 'dayna